In [1]:
import os
import glob
import numpy as np
import awkward

In [2]:
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt

In [3]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [4]:
# taken from the file_processor outputs -- update if needed
# v14 8gev
presel_eff = {1: 0.9988621022179364, 10: 0.999275930896827, 100: 0.9991549148445952, 1000: 0.9991183067080328, 0: 0.05252190640746514}

In [5]:
sig_filelist_b = glob.glob('/home/zwan/LDMX/LDMX-scripts/GraphNet/plot_data/ddp_lite_eat_ecal/v14_8gev_1*.parquet') + \
                 glob.glob('/home/zwan/LDMX/LDMX-scripts/GraphNet/plot_data/ddp_lite_eat_ecal/v14_8gev_0*.parquet')
bkg_filelist_b = glob.glob('/home/zwan/LDMX/LDMX-scripts/GraphNet/plot_data/ddp_lite_eat_ecal/*pn*.parquet')

In [6]:
sig_tables_b = [awkward.from_parquet(f) for f in sig_filelist_b]
bkg_tables_b = [awkward.from_parquet(f) for f in bkg_filelist_b]
#sig_tables_c = [awkward.from_parquet(f) for f in sig_filelist_c]
#bkg_tables_c = [awkward.from_parquet(f) for f in bkg_filelist_c]
"""print("Loaded")
print("sig_tables:")
print(awkward.type(sig_tables[0]))
print("bkg_tables:")
print(awkward.type(bkg_tables[0]))
print("Found {} bkg files".format(len(bkg_tables)))

sig_table = sig_tables[0]['ParticleNet_disc']
sig_class = sig_tables[0]['ParticleNet_extra_label']
sig_distr = [sig_table[i] for i in range(len(sig_table)) if sig_class[i] == 1000]
bkg_table = bkg_tables[0]['ParticleNet_disc']
bkg_distr = [bkg_table[i] for i in range(len(bkg_table))]
print("Arrs ready")
# Plot discValue distr, just to check...
#bins = np.linspace(0.996, 0.998, 100)
bins = np.linspace(0, 1, 100)
plt.figure()
plt.hist(sig_distr, bins=bins, density=True, histtype='step', label="sig")
plt.hist(bkg_distr, bins=bins, density=True, histtype='step', label="bkg")
#plt.xlim((0.996,0.998))
plt.xlabel("PN score")
plt.semilogy()
plt.legend(loc=1)
plt.title("PN score for 1.0 GeV")
plt.show()
print(sig_tables[0]['ParticleNet_disc'])"""

'print("Loaded")\nprint("sig_tables:")\nprint(awkward.type(sig_tables[0]))\nprint("bkg_tables:")\nprint(awkward.type(bkg_tables[0]))\nprint("Found {} bkg files".format(len(bkg_tables)))\n\nsig_table = sig_tables[0][\'ParticleNet_disc\']\nsig_class = sig_tables[0][\'ParticleNet_extra_label\']\nsig_distr = [sig_table[i] for i in range(len(sig_table)) if sig_class[i] == 1000]\nbkg_table = bkg_tables[0][\'ParticleNet_disc\']\nbkg_distr = [bkg_table[i] for i in range(len(bkg_table))]\nprint("Arrs ready")\n# Plot discValue distr, just to check...\n#bins = np.linspace(0.996, 0.998, 100)\nbins = np.linspace(0, 1, 100)\nplt.figure()\nplt.hist(sig_distr, bins=bins, density=True, histtype=\'step\', label="sig")\nplt.hist(bkg_distr, bins=bins, density=True, histtype=\'step\', label="bkg")\n#plt.xlim((0.996,0.998))\nplt.xlabel("PN score")\nplt.semilogy()\nplt.legend(loc=1)\nplt.title("PN score for 1.0 GeV")\nplt.show()\nprint(sig_tables[0][\'ParticleNet_disc\'])'

In [7]:
load_branches = [
    'discValue_',
    'recoilX_',
    'recoilY_',
     
    'ParticleNet_extra_label',
    'ParticleNet_disc',
    'TargetSPRecoilE_pt', # use this for plotting: this is the recoil electron pT at TargetSP
    'maxPE'
]

In [8]:
"""
a = {}
for k in load_branches:
    print("Loading "+k)
    arrs = []
    for tab in sig_tables + bkg_tables:
        #print(awkward.type(tab))
        #print(tab)
        #print(tab['TargetSPRecoilE_pt'])
        #print(awkward.fields(tab))
        arr = tab[k] if k in awkward.fields(tab) else np.zeros_like(tab['ParticleNet_disc'])
        arrs.append(arr)
    a[k] = awkward.concatenate(arrs)
    #if k.startswith('EcalVeto'):
    #    #a[k] = a[k].regular()
    #    a[k] = awkward.to_regular(a[k])
    if a[k].ndim==2 and len(a[k][0]) == 1:  #a[k].shape[1]==1:
        a[k] = a[k][:,0]
    #print("   Found {} events".format(len(a[k])))

"""
def load_dict(sig_tables, bkg_tables):
    a = {}
    for k in load_branches:
        #print("Loading "+k)
        arrs = []
        for tab in sig_tables + bkg_tables:
            #print(awkward.type(tab))
            #print(tab)
            #print(tab['TargetSPRecoilE_pt'])
            #print(awkward.fields(tab))
            arr = tab[k] if k in awkward.fields(tab) else np.zeros_like(tab['ParticleNet_disc'])
            arrs.append(arr)
        a[k] = awkward.concatenate(arrs)
        if k.startswith('EcalVeto'):
            #a[k] = a[k].regular()
            a[k] = awkward.to_regular(a[k])
            if a[k].ndim==2 and len(a[k][0]) == 1:  #a[k].shape[1]==1:
                a[k] = a[k][:,0]
        print("   Found {} events".format(len(a[k])))
    return a

b = load_dict(sig_tables_b, bkg_tables_b)
#c = load_dict(sig_tables_c, bkg_tables_c)

#t_extra_labels = a['ParticleNet_extra_label']
#print("TESTING")
#print(a['TargetSPRecoilE_pt'][:100])
#print("DONE")
"""p_rec = a['EcalVeto_v12.recoilX_']
#NEW for finding frac of missing-e events
for k in presel_eff:  #Find the fraction of missing-e events for each signal category
    n_events=0
    n_missing_e=0
    for i in range(len(p_rec)):
        if t_extra_labels[i] == k:
            n_events += 1
            #if i<100:  print(p_rec[i])
            if p_rec[i] == -9999:  n_missing_e += 1
    print("Found", n_events, "events for", k, "MeV")
    print("Fraction of events w/ missing electron for "+str(k)+" MeV is ", str(n_missing_e/n_events))"""
print("Done")

   Found 395652 events
   Found 395652 events
   Found 395652 events
   Found 395652 events
   Found 395652 events
   Found 395652 events
   Found 395652 events
Done


In [9]:
#NEW:  Remove all hits with/without recoilX_==-999 (or -9999?) (or Y)
"""ntotal = len(a['EcalVeto_v12.recoilX_'])
mask = a['EcalVeto_v12.recoilX_'] != -9999
#print(mask[:50])
for k in load_branches:
    a[k] = a[k][mask]
print("Fraction of events remaining (without missing electron):", len(a[load_branches[0]])/len(mask))"""

#Examine missing electrons in each category:
#NOTE:  uncertain how bkg_tables, etc are structured...
#print(sig_tables[0])

'ntotal = len(a[\'EcalVeto_v12.recoilX_\'])\nmask = a[\'EcalVeto_v12.recoilX_\'] != -9999\n#print(mask[:50])\nfor k in load_branches:\n    a[k] = a[k][mask]\nprint("Fraction of events remaining (without missing electron):", len(a[load_branches[0]])/len(mask))'

In [10]:
def to_categorical(y, num_classes=None):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
    # Returns
        A binary matrix representation of the input.
    """
    y = np.array(y, dtype='int').ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype='int')
    categorical[np.arange(n), y] = 1
    return categorical


def plotROC(y_preds, y_truth, sample_weight=None, output=None, labels=['signal'], sig_eff=1, bkg_eff=1, energy=0, **kwargs):
    from sklearn.metrics import auc, roc_curve, accuracy_score

    fpr = dict()
    tpr = dict()
    thresholds= dict()
    roc_auc = dict()
    outputs = {}

    plt.figure()

    for label, pred in zip(labels, y_preds):
        print("Label: ", label)
        fpr[label], tpr[label], thresholds[label] = roc_curve(y_truth, pred, sample_weight=sample_weight)
        roc_auc[label] = auc(fpr[label], tpr[label])
        fpr[label] *= bkg_eff
        tpr[label] *= sig_eff

        legend = '%s (auc* = %0.6f)' % (label, roc_auc[label])
        print(legend)
        eff = get_signal_effs(fpr[label], tpr[label], thresholds[label])
        outputs[label] = eff
        print(eff)
        plt.plot(fpr[label], tpr[label], label=legend)
#     plt.plot([0, 1], [1, 0], 'k--')
    plt.xlim(kwargs.get('xlim', [0, 1]))
    plt.ylim(kwargs.get('ylim', [0, 1]))
    plt.xlabel('False positive rate ($\epsilon_{B}$)')
    plt.ylabel('True positive rate ($\epsilon_{S}$)')
#     plt.title('Receiver operating characteristic example')
    plt.legend(loc='best')
    if kwargs.get('logy', False):
        plt.yscale('log')
    if kwargs.get('logx', False):
        plt.xscale('log')
    plt.grid()
    # TEMPORARY TITLE
    plt.title(str(k)+" MeV", fontdict = {'fontsize' : 15})
    #plt.title("All events, "+str(k)+" MeV", fontdict = {'fontsize' : 15})
    if output:
        plt.savefig(output)
#     return {'fpr':fpr, 'tpr':tpr, 'thresholds':thresholds}
    return outputs


def plotROC_multi(y_preds_, y_truth_, sample_weight_=None, output=None, labels=['signal'], sig_eff=1, bkg_eff=1, energy=0, **kwargs):
    from sklearn.metrics import auc, roc_curve, accuracy_score

    # y_preds, etc are now tuples of (a, b, c) (1reg, 2reg, 3reg)
    plt.figure()
    
    for i in range(1):  # 1, 2, 3 regions
        
        fpr = dict()
        tpr = dict()
        thresholds= dict()
        roc_auc = dict()
        outputs = {}

        #plt.figure()

        y_preds = y_preds_[i]
        y_truth = y_truth_[i]
        sample_weight = sample_weight_[i]
        print("Y_TRUTH:")
        print(y_truth[:10])
        for label, pred in zip(labels, y_preds):
            if label == 'BDT' and i < 1:  continue
            print("Label: ", label)
            fpr[label], tpr[label], thresholds[label] = roc_curve(y_truth, pred, sample_weight=sample_weight)
            roc_auc[label] = auc(fpr[label], tpr[label])
            fpr[label] *= bkg_eff
            tpr[label] *= sig_eff
            
            if label == 'BDT':
                legend = '%s\n(auc* = %0.6f)' % (label+' (old Gabriel)', roc_auc[label])
            elif i == 0:
                legend = '(1,1)-reg SplitNetX\n(auc* = %0.6f)' % (roc_auc[label])
            #else:
                #legend = '%i-reg %s, PN (auc* = %0.6f)' % (i+2, label, roc_auc[label])
            else:
                legend = '1-reg SplitNet\n(auc* = %0.6f)' % (roc_auc[label])
            print(legend)
            eff = get_signal_effs(fpr[label], tpr[label], thresholds[label])
            outputs[label] = eff
            print(eff)
            print(len(eff))
            #print(tpr[label][:10])
            #print(len(fpr[label]))
            plt.plot(fpr[label], tpr[label], label=legend)
#     plt.plot([0, 1], [1, 0], 'k--')
    plt.xlim(kwargs.get('xlim', [0, 1]))
    plt.ylim(kwargs.get('ylim', [0, 1]))
    plt.xlabel('False positive rate ($\epsilon_{B}$)', fontsize=12)
    plt.ylabel('True positive rate ($\epsilon_{S}$)', fontsize=12)
#     plt.title('Receiver operating characteristic example')
    plt.legend(loc='best')
    if kwargs.get('logy', False):
        plt.yscale('log')
    if kwargs.get('logx', False):
        plt.xscale('log')
    plt.grid()
    # TEMPORARY TITLE
    plt.title(str(k)+" MeV w/ PN Background", fontdict = {'fontsize' : 15})
    #plt.title("All events, "+str(k)+" MeV", fontdict = {'fontsize' : 15})
    if output:
        plt.savefig(output, facecolor='w', dpi=250)
#     return {'fpr':fpr, 'tpr':tpr, 'thresholds':thresholds}
    return outputs


mistags=[1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9]
def get_signal_effs(fpr, tpr, thresholds, mistags=mistags):
    outputs = []
    for m in mistags:
        idx = next(idx for idx, v in enumerate(fpr) if v > m)
        outputs.append((fpr[idx], tpr[idx], thresholds[idx]))
    return outputs

In [11]:
test_extra_labels_b = b['ParticleNet_extra_label']
test_labels_b = test_extra_labels_b>0
#test_extra_labels_c = c['ParticleNet_extra_label']
#test_labels_c = test_extra_labels_c>0
#print(test_labels_b[-20:])
print(len(test_extra_labels_b))
print(test_extra_labels_b[-20:])

395652
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
"""
roc_info = {}
for k in presel_eff:
    if k > 0:
        mass = '%d MeV' % k
        print(mass)
        roc_info[k] = plotROC([a['ParticleNet_disc'], a['discValue_']], test_labels, 
                sample_weight=np.logical_or(test_extra_labels == 0, test_extra_labels == k),
                sig_eff=presel_eff[k], bkg_eff=presel_eff[0],
                labels=['SplitNet', 'BDT'], xlim=[1e-6, .01], ylim=[0, 1], logx=True, energy=k)
        #**NOTE:**  The sample_weight line is critical!!  Selects only bkg+sig[k].
"""
roc_info = {}
#y_preds_ = [[a['ParticleNet_disc'], a['discValue_']], [b['ParticleNet_disc'], b['discValue_']], [c['ParticleNet_disc'], c['discValue_']]]
y_preds_ = [[b['ParticleNet_disc'], b['discValue_']]]
#test_labels_ = [test_labels, test_labels_b, test_labels_c]
test_labels_ = [test_labels_b]
#print(a['ParticleNet_disc'][:10])
for k in presel_eff:
    if k > 0:
        mass = '%d MeV' % k
        print(mass)
        weights = [np.logical_or(test_extra_labels_b==0, test_extra_labels_b==k)]
                  #np.logical_or(test_extra_labels_c==0, test_extra_labels_c==k)]
        print(type(weights))
        roc_info[k] = plotROC_multi(y_preds_, test_labels_, 
                sample_weight_=weights,
                sig_eff=presel_eff[k], bkg_eff=presel_eff[0],
                labels=['SplitNet'], xlim=[1e-9, 0.1], ylim=[0, 1], logx=True, energy=k, output=f'v14_pnx_ROC_{mass}')
# NOTE:  add 'BDT' to labels list if desired

1 MeV
<class 'list'>


<IPython.core.display.Javascript object>

Y_TRUTH:
[True, True, True, True, True, True, True, True, True, True]
Label:  SplitNet
(1,1)-reg SplitNetX
(auc* = 0.837786)
[(0.0010003718244396673, 0.0784946306304464, 0.8734933594907704), (0.00010019622406947382, 0.005825040755147674, 0.9097976705444546), (1.1132913785497092e-05, 0.0009377433033855157, 0.9324455875746115), (1.5904162550710132e-06, 5.5161370787383276e-05, 0.9757410186394541), (1.5904162550710132e-06, 5.5161370787383276e-05, 0.9757410186394541), (1.5904162550710132e-06, 5.5161370787383276e-05, 0.9757410186394541), (1.5904162550710132e-06, 5.5161370787383276e-05, 0.9757410186394541)]
7
10 MeV
<class 'list'>


<IPython.core.display.Javascript object>

Y_TRUTH:
[True, True, True, True, True, True, True, True, True, True]
Label:  SplitNet
(1,1)-reg SplitNetX
(auc* = 0.808700)
[(0.0010019622406947381, 0.060033912326741656, 0.8734888283844316), (0.00010019622406947382, 0.004540009522506895, 0.9097976705444546), (1.1132913785497092e-05, 0.0009256330094431532, 0.9324455875746115), (1.5904162550710132e-06, 5.5097202943044834e-05, 0.9757410186394541), (1.5904162550710132e-06, 5.5097202943044834e-05, 0.9757410186394541), (1.5904162550710132e-06, 5.5097202943044834e-05, 0.9757410186394541), (1.5904162550710132e-06, 5.5097202943044834e-05, 0.9757410186394541)]
7
100 MeV
<class 'list'>


<IPython.core.display.Javascript object>

Y_TRUTH:
[True, True, True, True, True, True, True, True, True, True]
Label:  SplitNet
(1,1)-reg SplitNetX
(auc* = 0.799481)
[(0.0010003718244396673, 0.0579688131430366, 0.8734933594907704), (0.00010019622406947382, 0.004440492624338291, 0.9097976705444546), (1.1132913785497092e-05, 0.0008153758168760137, 0.9324455875746115), (1.5904162550710132e-06, 3.305577635983839e-05, 0.9757410186394541), (1.5904162550710132e-06, 3.305577635983839e-05, 0.9757410186394541), (1.5904162550710132e-06, 3.305577635983839e-05, 0.9757410186394541), (1.5904162550710132e-06, 3.305577635983839e-05, 0.9757410186394541)]
7
1000 MeV
<class 'list'>


<IPython.core.display.Javascript object>

Y_TRUTH:
[True, True, True, True, True, True, True, True, True, True]
Label:  SplitNet
(1,1)-reg SplitNetX
(auc* = 0.547133)
[(0.0010019622406947381, 0.015076085817552817, 0.8734888283844316), (0.00010019622406947382, 0.0008479609992341615, 0.9097976705444546), (1.1132913785497092e-05, 0.0002422745712097604, 0.9324455875746115), (1.5904162550710132e-06, 1.1012480509534564e-05, 0.9757410186394541), (1.5904162550710132e-06, 1.1012480509534564e-05, 0.9757410186394541), (1.5904162550710132e-06, 1.1012480509534564e-05, 0.9757410186394541), (1.5904162550710132e-06, 1.1012480509534564e-05, 0.9757410186394541)]
7


In [13]:
disc_threshold_b = 0.95

bkg_pt_b = np.array(b['TargetSPRecoilE_pt'][test_extra_labels_b==0])
bkg_disc_value_b = b['ParticleNet_disc'][test_extra_labels_b==0]
bkg_maxPE_b = b['maxPE'][test_extra_labels_b==0]

nPass_b = np.sum( (bkg_disc_value_b > disc_threshold_b) * (bkg_pt_b.T[0] != -999) )
nPass_veto_b = np.sum( (bkg_disc_value_b > disc_threshold_b) * (bkg_pt_b.T[0] != -999) * (bkg_maxPE_b < 8) )

print(nPass_b)
print(nPass_veto_b)

2
0


In [14]:
masses = [0, 1, 10, 100, 1000]
pT_b = {}
nEvents_b = {}
for m in masses:
    pT_b[m] = np.array(b['TargetSPRecoilE_pt'][test_extra_labels_b==m]).T[0]
    print(len(pT_b[m]))
    pT_b[m] = pT_b[m][pT_b[m] != -999]
    nEvents_b[m] = len(pT_b[m])
    print(len(pT_b[m]))
print(nEvents_b)

33024
33024
90540
90540
90683
90683
90679
90679
90726
90726
{0: 33024, 1: 90540, 10: 90683, 100: 90679, 1000: 90726}


In [15]:
thresholds = [0.5, 0.73, 0.88, 0.95] #(24k, 10k, 1k, 100)
pT_pass_b = {}
for m in masses:
    disc_value = np.array(b['ParticleNet_disc'][test_extra_labels_b==m])
    print(disc_value.shape)
    pt = np.array(b['TargetSPRecoilE_pt'][test_extra_labels_b==m])
    print(pt.T[0])
    print(pt.shape)
    disc_value = disc_value[pt.T[0]!=-999]
    pT_pass_b[m] = [pT_b[m][disc_value > threshold] for threshold in thresholds]
    print(len(pT_pass_b[m]))

(33024,)
[10.64500046  7.82665873  3.92082477 ...  7.62220192  6.82020617
  3.13451433]
(33024, 1)
4
(90540,)
[10.64109802  7.26533222  4.54243231 ...  3.98871088 18.61163712
 24.40962601]
(90540, 1)
4
(90683,)
[ 7.43232298 12.86582661 15.13844585 ...  3.0825727  15.4311409
  9.59808064]
(90683, 1)
4
(90679,)
[11.92726231  5.67457104  6.92010927 ...  9.44293118  7.02296066
  3.99294686]
(90679, 1)
4
(90726,)
[11.79983521  5.96989679  7.52992058 ...  2.78571105  4.74370098
  0.92160106]
(90726, 1)
4


In [16]:
print(pT_pass_b)

{0: [array([ 7.82665873,  3.92082477, 11.53840733, ...,  2.77211285,
        7.62220192,  3.13451433]), array([ 4.12045956,  8.66702557,  8.14262295, ..., 10.57321548,
        7.02946043,  3.13451433]), array([ 4.12045956, 10.30980873,  2.63635612,  4.89313745,  5.04441833,
        1.50318432,  7.74153233,  8.21622753,  9.88457394,  5.88651514,
        3.16193986,  9.63127232, 12.67311192, 14.5062685 ,  5.5068469 ,
        7.19772005,  4.46213675, 11.25712585,  1.31201792, 15.68619537,
        6.45615578, 15.35877895,  3.67673302,  8.30364799, 11.29188347,
        4.95684528,  2.2682941 ,  8.59948635,  6.47326708,  5.26358128,
        5.07345295, 10.11571407,  5.00075483,  6.59914207,  5.80858898,
        0.80640382,  3.23539639, 12.67164993,  9.86852741,  9.41983414,
       11.23495388, 12.28436279,  4.00205469,  2.13387799,  7.13818312,
        8.92088032, 10.00003624, 13.91774082,  7.42738104,  0.42379722,
       10.05321789,  5.32375288,  2.52289081,  6.92203856,  6.31350851,
     

In [17]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

colors = ['#4A7DFF', '#14AD0C', '#FF212E', '#FF8F13', '#871EFE']

for m in masses:
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, gridspec_kw={'height_ratios': [2, 1]})
    #if m == 1000:
        #ax1.set_ylim(100, 1300)
    vals = [pT_b[m], pT_pass_b[m]]
    bins = np.linspace(-50, 200, 51)
    labels = ['N bkg ~ $2 x 10^{4}$', 'N bkg ~ $10^{4}$', 'N bkg ~ $10^{3}$', 'N bkg ~ $10^{2}$']
    n1, bins, _ = ax1.hist(vals[0], bins=bins, range=(0, 250), density=False, stacked=False, histtype='step', color=colors[-1],
                log=True, label='Inclusive')
    n2, bins, _ = ax1.hist(vals[1], bins=bins, range=(0, 250), density=False, stacked=False, histtype='step', color=colors[:-1],
                    log=True, label=labels)
    #n1, bins, _ = ax1.hist(vals[0], bins=bins, range=(0, 250), density=False, stacked=False, histtype='step', color=colors[0], 
                 #log=True, label='Inclusive')
    #n2, bins, _ = ax1.hist(vals[1], bins=bins, range=(0, 250), density=False, stacked=False, histtype='step', color=colors[3], 
                 #log=True, label=f'ParticleNetX_disc > {disc_threshold}')
    ax1.legend()
    
    ratio_arrs = []
    errs = []
    for val in n2:
        ratio_arr = val / n1
        err = (val / n1) * ( (1/np.sqrt(val)) + (1/np.sqrt(n1)) ) 
        #print(type(ratio_arr))
        #print(len(ratio_arr))       
        #print(len(bins[:-1]))
        ratio_arrs.append(ratio_arr)
        errs.append(err)
    
    for i, ratio_arr in enumerate(ratio_arrs):

        ax2.errorbar(bins[:-1],     
            ratio_arr,
            yerr=errs[i],
            fmt='o',
            alpha=0.8,
            color=colors[i])

    '''
    ax2.errorbar(bins[:-1],     # this is what makes it comparable
        n2 / n1, # maybe check for div-by-zero!
        alpha=0.8,
        yerr = (n2 / n1) * ( (1/np.sqrt(n2)) + (1/np.sqrt(n1)) ),
        fmt='o',
        color=colors[1]        
                            )
    '''
    
    ax1.set_xlim(-10, 210)
    ax2.set_xlim(-10, 210)
    ax2.set_ylim(-0.05, 1.05)
    ax2.grid()
    ax2.set_xlabel("pT [MeV]", fontsize=14)
    
    ax1.set_ylabel('Number of Events', fontsize=13)
    
    ax1.set_title(f'{m} MeV TargetSP Recoil $e^{{-}}$ Transverse Momentum' if m!=0 else 'PN Bkg TargetSP Recoil $e^{{-}}$ Transverse Momentum', fontsize=15)
    ax2.set_ylabel('Ratio', fontsize=13)
    plt.tight_layout()
    plt.show()
    plt.savefig(f'pT_bias_{m}', facecolor='w', dpi=250)

print("done")

<IPython.core.display.Javascript object>

/tmp/ipykernel_137916/368068625.py:26: RuntimeWarning: invalid value encountered in divide
  ratio_arr = val / n1
/tmp/ipykernel_137916/368068625.py:27: RuntimeWarning: invalid value encountered in divide
  err = (val / n1) * ( (1/np.sqrt(val)) + (1/np.sqrt(n1)) )
/tmp/ipykernel_137916/368068625.py:27: RuntimeWarning: divide by zero encountered in divide
  err = (val / n1) * ( (1/np.sqrt(val)) + (1/np.sqrt(n1)) )
/tmp/ipykernel_137916/368068625.py:27: RuntimeWarning: invalid value encountered in multiply
  err = (val / n1) * ( (1/np.sqrt(val)) + (1/np.sqrt(n1)) )


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

done


In [18]:
pn_sig_disc = {}

for m in masses[1:]:
    pn_sig_disc[m] = b['ParticleNet_disc'][test_extra_labels_b==m]

pn_sig_pass = {}
for m in masses[1:]:
    pn_sig_pass[m] = np.sum( (pn_sig_disc[m] > disc_threshold_b) * (b['TargetSPRecoilE_pt'][test_extra_labels_b==m] != -999) * (b['maxPE'][test_extra_labels_b==m] < 8) )

pn_sig_eff = {}
for m in masses[1:]:
    pn_sig_eff[m] = (pn_sig_pass[m] / nEvents_b[m]) * presel_eff[m]
    
print(pn_sig_eff)

{1: 0.0003420004988817763, 10: 0.00036364153942409594, 100: 0.0003525949478382762, 1000: 8.809984407627651e-05}


In [19]:
maxPE = a['maxPE'][test_extra_labels==0]
print(np.sum(maxPE<8))

NameError: name 'a' is not defined